In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)y
import random
import json
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/router-placement-llm/Defaults.txt
/kaggle/input/router-placement-llm/Information.txt
/kaggle/input/router-placement-llm/Environment/Office.txt
/kaggle/input/router-placement-llm/Environment/Banquet.txt
/kaggle/input/router-placement-llm/Environment/House.txt
/kaggle/input/router-placement-llm/Environment/Warehouse.txt
/kaggle/input/router-placement-llm/Environment/Hospital.txt
/kaggle/input/router-placement-llm/Dataset_json/Parsed_User_Requests_KHM.json
/kaggle/input/router-placement-llm/Dataset_json/Parsed_User_Requests_KM.json
/kaggle/input/router-placement-llm/Dataset_json/Parsed_User_Requests_WKHM.json
/kaggle/input/router-placement-llm/Dataset_json/Parsed_User_Requests_KC.json
/kaggle/input/router-placement-llm/Dataset_json/Parsed_User_Requests_CKM.json
/kaggle/input/router-placement-llm/Routers/Nokia-7750.txt
/kaggle/input/router-placement-llm/Routers/Cisco-XR-1234.txt
/kaggle/input/router-placement-llm/Routers/Huawei-NE-40.txt
/kaggle/input/router-placement-llm/Rou

In [3]:

# Path to the Dataset_json folder in the Kaggle dataset
dataset_json_path = "/kaggle/input/router-placement-llm/Dataset_json"
dataset_list = []

# Iterate through files in the Dataset_json folder
for file_name in os.listdir(dataset_json_path):
    file_path = os.path.join(dataset_json_path, file_name)
    
    # Check if the file is a JSON file
    if file_name.endswith(".json"):
        try:
            # Read and load JSON content
            with open(file_path, 'r') as file:
                data = json.load(file)
                dataset_list.append(data)
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

# # Print the list or process it further
# print(dataset_list)


In [4]:
# Path to the Dataset_json folder in the Kaggle dataset
dataset_json_path = "/kaggle/input/router-placement-llm/Environment"
environment_list = []

# Iterate through files in the Dataset_json folder
for file_name in os.listdir(dataset_json_path):
    file_path = os.path.join(dataset_json_path, file_name)
    
    with open(file_path, 'r') as file:
        data = file.read()
    file.close()
    environment_list.append(data)

In [5]:
# Path to the Dataset_json folder in the Kaggle dataset
dataset_json_path = "/kaggle/input/router-placement-llm/Routers"
router_list = []

# Iterate through files in the Dataset_json folder
for file_name in os.listdir(dataset_json_path):
    file_path = os.path.join(dataset_json_path, file_name)
    
    with open(file_path, 'r') as file:
        data = file.read()
    file.close()
    router_list.append(data)

In [6]:
with open("/kaggle/input/router-placement-llm/Defaults.txt",'r') as file:
    defaults=file.read()

In [7]:
print(defaults)

# Default Parameter Values for Indoor Wireless Communication Modeling  

This document outlines the default parameter values to be used for indoor wireless communication simulations. These parameters are optimized for typical indoor environments and device configurations.  

---

### Default Parameter Values  

| **Parameter**               | **Default Value**             |  
|-----------------------------|-------------------------------|  
| **X User Distribution**     | Uniform                      |  
| **Y User Distribution**     | Uniform                      |  
| **Height**                  | \( 3.0 \, \text{m} \)        |  
| **Path Loss Parameter \( K \)** | \( 0.005 \, \text{dB} \)  |  
| **Path Loss Exponent \( \alpha \)** | \( 3.0 \)             |  
| **Device Name**             | Cisco ASR-900                |  
| **User Equipment Height**   | \( 1.0 \, \text{m} \)        |  

---

### Parameter Details  

1. **User Distribution**:  
   - **Along the Edge**: Refers to **ex

In [8]:
with open("/kaggle/input/router-placement-llm/Information.txt",'r') as file:
    information= json.load(file)

In [9]:
print(information)

{'environments': [{'name': 'Hospital', 'K': 0.015, 'alpha': 3.0, 'user_distribution': 'Along the Edge'}, {'name': 'House', 'K': 0.02, 'alpha': 2.8, 'user_distribution': 'Gaussian Distribution'}, {'name': 'Office', 'K': 0.03, 'alpha': 2.5, 'user_distribution': 'Spread Evenly'}, {'name': 'Warehouse', 'K': 0.03, 'alpha': 3.2, 'user_distribution': 'Uniform Distribution'}, {'name': 'Banquet Hall', 'K': 0.02, 'alpha': 2.8, 'user_distribution': 'Concentrated in the Center'}], 'routers': [{'model': 'Huawei NE40E', 'transmission_power_2.4GHz': 22, 'carrier_frequency_band': '2400–2483.5 MHz'}, {'model': 'Juniper MX240', 'transmission_power_2.4GHz': 24, 'carrier_frequency_band': '2378–2457.1 MHz'}, {'model': 'Nokia 7750', 'transmission_power_2.4GHz': 20, 'carrier_frequency_band': '2200 MHz–2500 MHz'}, {'model': 'Cisco ASR-900', 'transmission_power_2.4GHz': 23, 'carrier_frequency_band': '2350–2450 MHz'}, {'model': 'Cisco XR-1234', 'transmission_power_2.4GHz': 20, 'carrier_frequency_band': '2301–24

In [10]:
!pip install torch transformers transformer_lens 
!pip install -i https://pypi.org/simple/ bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 14.2 MB/s eta 0:00:0000:0100:01


In [ ]:
HF_token ="<token>"

In [12]:
pip install -U jupyterlab ipywidgets jupyterlab-widgets;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 96.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
  Attempting uninstall:

In [13]:
use_8bit = True

bnb_config = BitsAndBytesConfig(
    load_in_8bit=use_8bit,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

# Load the Llama 3 8B model and tokenizer
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_token)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [14]:
llama_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                   quantization_config=bnb_config,
                                                   device_map="auto",
                                                   token=HF_token)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [15]:
# del llama_model
# import gc
# import torch

# # Assuming `obj` is the object consuming GPU memory
# obj = None

# # Collect garbage
# gc.collect()

# # Empty PyTorch cache
# torch.cuda.empty_cache()

In [16]:
system={"role": "system", 
    "content": 
"""
You need to provide a jsonised output containing the following sections extracted from the query:
{
"Parsed_Parameters": {
                "X_dimension": <float>,
                "Y_dimension": <float>,
                "Number_of_Users": <int>,
                "Number_of_Routers": <int>,
                "X_User_Distribution": <string>,
                "Y_User_Distribution": <string>,
                "Height": <float>,
                "K": <float>,
                "Alpha": <float>,
                "Transmit_Power": <float>,
                "User Equipment Height":<float>
            }
}

Example of what your output should look like:
{
  "Parsed_Parameters": {
    "X_dimension": 30.0,
    "Y_dimension": 40.0,
    "Number_of_Users": 100,
    "Number_of_Routers": 6,
    "X_User_Distribution": "Exponential",
    "Y_User_Distribution": "Exponential",
    "Height": 4.0,
    "K": 15.0,
    "Alpha": 0.005,
    "Transmit_Power": 19,
    "User Equipment Height": 1.0
  }
}

Any information missing in the user request must be retreived from router specification files or environment files in the context files provided.

X and Y Distribution must be one of Uniform, Gaussian, Exponential, Bi-Exponential

Only provide the output parsed json without any reasoning or any other statements.

Extract only the parameters mentioned in the example output.
"""
}

In [19]:
context='CONTEXT: \n'
for f in environment_list:
    context=context+f
for f in router_list:
    context=context+f
context=context+defaults

In [20]:
print(dataset_list[0]["2"][11]["Original_Request"])

Can you optimize the placement of 8 routers in a 50m x 20m office with 180 users to reduce power degradation? The users are concentrated near one corner and the device is a Cisco ASR-900.


In [21]:
request=dataset_list[0]["2"][11]["Original_Request"]
message=[system,
    {"role":"user","content":context+"\n User Request: \n " + request}]
input_ids = tokenizer.apply_chat_template(message, return_tensors="pt",padding=True, truncation=True,).to(llama_model.device)
with torch.no_grad():
     output = llama_model.generate(input_ids, max_length=input_ids.shape[1]+500, num_return_sequences=1, temperature=0.1);
input_length = input_ids.shape[1] 
clean_prompt=output[0][input_length+3:];
answer=tokenizer.decode(clean_prompt, skip_special_tokens=True);


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [22]:
print(answer)



{
  "Parsed_Parameters": {
    "X_dimension": 50.0,
    "Y_dimension": 20.0,
    "Number_of_Users": 180,
    "Number_of_Routers": 8,
    "X_User_Distribution": "Along One Corner",
    "Y_User_Distribution": "Uniform",
    "Height": 3.0,
    "K": 0.005,
    "Alpha": 3.0,
    "Transmit_Power": 19,
    "User_Equipment_Height": 1.0
  }
}


In [23]:
json_answers=[]

In [58]:
import gc
# Collect garbage
gc.collect()

# Empty PyTorch cache
torch.cuda.empty_cache()

In [49]:
for cls in range(0,5):
    print("Class",cls)
    for typ in range(3):
        print("Type",typ)
        for ins in range(10):
            count+=1
            request=dataset_list[cls][str(typ)][ins]["Original_Request"]
            message=[system,
                {"role":"user","content":context+"\n User Request: \n " + request}]
            input_ids = tokenizer.apply_chat_template(message, return_tensors="pt",padding=True, truncation=True,).to(llama_model.device)
            with torch.no_grad():
                 output = llama_model.generate(input_ids, max_length=input_ids.shape[1]+500, num_return_sequences=1, temperature=0.1);
            input_length = input_ids.shape[1] 
            clean_prompt=output[0][input_length+3:];
            answer=tokenizer.decode(clean_prompt, skip_special_tokens=True);
            print(count,":")
            print(answer)
            print(",")
            new_answers.append(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Class 4
Type 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


31 :


{
  "Parsed_Parameters": {
    "X_dimension": 40.0,
    "Y_dimension": 30.0,
    "Number_of_Users": 120,
    "Number_of_Routers": 5,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 4.0,
    "K": 0.02,
    "Alpha": 2.5,
    "Transmit_Power": 20,
    "User_Equipment_Height": 1.0
  }
}
,


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

57 :


{
  "Parsed_Parameters": {
    "X_dimension": 10.0,
    "Y_dimension": 20.0,
    "Number_of_Users": 40,
    "Number_of_Routers": 3,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 3.0,
    "K": 0.005,
    "Alpha": 3.0,
    "Transmit_Power": 19,
    "User_Equipment_Height": 1.0
  }
}
,


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


58 :


{
  "Parsed_Parameters": {
    "X_dimension": 40.0,
    "Y_dimension": 60.0,
    "Number_of_Users": 180,
    "Number_of_Routers": 8,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 5.0,
    "K": 0.03,
    "Alpha": 3.2,
    "Transmit_Power": 20,
    "User_Equipment_Height": 1.5
  }
}
,


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


59 :


{
  "Parsed_Parameters": {
    "X_dimension": 25.0,
    "Y_dimension": 40.0,
    "Number_of_Users": 120,
    "Number_of_Routers": 6,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 3.0,
    "K": 0.005,
    "Alpha": 3.0,
    "Transmit_Power": 19,
    "User_Equipment_Height": 1.0
  }
}
,
60 :


{
  "Parsed_Parameters": {
    "X_dimension": 20.0,
    "Y_dimension": 30.0,
    "Number_of_Users": 90,
    "Number_of_Routers": 5,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 3.0,
    "K": 0.005,
    "Alpha": 3.0,
    "Transmit_Power": 19,
    "User_Equipment_Height": 1.0
  }
}
,


In [60]:
system2={"role": "system", 
    "content": 
"""
You need to provide a jsonised output containing the following sections extracted from the query:
{
"Parsed_Parameters": {
                "X_dimension": <float>,
                "Y_dimension": <float>,
                "Number_of_Users": <int>,
                "Number_of_Routers": <int>,
                "X_User_Distribution": <string>,
                "Y_User_Distribution": <string>,
                "Height": <float>,
                "K": <float>,
                "Alpha": <float>,
                "Transmit_Power": <float>,
                "User Equipment Height":<float>
            }
}

Example of what your output should look like:
{
  "Parsed_Parameters": {
    "X_dimension": 30.0,
    "Y_dimension": 40.0,
    "Number_of_Users": 100,
    "Number_of_Routers": 6,
    "X_User_Distribution": "Exponential",
    "Y_User_Distribution": "Exponential",
    "Height": 4.0,
    "K": 15.0,
    "Alpha": 0.005,
    "Transmit_Power": 19,
    "User Equipment Height": 1.0
  }
}

X and Y Distribution must be one of Uniform, Gaussian, Exponential, Bi-Exponential

Only provide the output parsed json without any reasoning or any other statements.

Extract only the parameters mentioned in the example output.
"""
}

In [66]:
no_context=[]

In [67]:
count=0

In [68]:
import time

In [69]:
for cls in range(0,5):
    print("Class",cls)
    for typ in range(3):
        print("Type",typ)
        for ins in range(10):
            count+=1
            request=dataset_list[cls][str(typ)][ins]["Original_Request"]
            message=[system2,
                {"role":"user","content": "User Request: \n " + request}]
            input_ids = tokenizer.apply_chat_template(message, return_tensors="pt",padding=True, truncation=True,).to(llama_model.device)
            start=time.time()
            with torch.no_grad():
                 output = llama_model.generate(input_ids, max_length=input_ids.shape[1]+500, num_return_sequences=1, temperature=0.1);
            input_length = input_ids.shape[1] 
            clean_prompt=output[0][input_length+3:];
            answer=tokenizer.decode(clean_prompt, skip_special_tokens=True);
            print(count,":")
            print(answer)
            print(",")
            print(time.time()-start)
            no_context.append(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Class 0
Type 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1 :


{
  "Parsed_Parameters": {
    "X_dimension": 30.0,
    "Y_dimension": 40.0,
    "Number_of_Users": 100,
    "Number_of_Routers": 6,
    "X_User_Distribution": "Exponential",
    "Y_User_Distribution": "Exponential",
    "Height": 4.0,
    "K": 0.005,
    "Alpha": 2.0,
    "Transmit_Power": 15,
    "User_Equipment_Height": 1.0
  }
}
,
34.83257341384888


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

147 :


{
  "Parsed_Parameters": {
    "X_dimension": 10.0,
    "Y_dimension": 20.0,
    "Number_of_Users": 40,
    "Number_of_Routers": 3,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 0.0,
    "K": 0.0,
    "Alpha": 0.0,
    "Transmit_Power": 0.0,
    "User_Equipment_Height": 0.0
  }
}
,
34.58257555961609


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


148 :


{
  "Parsed_Parameters": {
    "X_dimension": 60.0,
    "Y_dimension": 40.0,
    "Number_of_Users": 180,
    "Number_of_Routers": 8,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 4.0,
    "K": 15.0,
    "Alpha": 0.005,
    "Transmit_Power": 19,
    "User_Equipment_Height": 1.0
  }
}
,
34.0470757484436


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


149 :


{
  "Parsed_Parameters": {
    "X_dimension": 25.0,
    "Y_dimension": 40.0,
    "Number_of_Users": 120,
    "Number_of_Routers": 6,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 1.0,
    "K": 0.0,
    "Alpha": 0.0,
    "Transmit_Power": 0.0,
    "User_Equipment_Height": 1.0
  }
}
,
34.61381125450134
150 :


{
  "Parsed_Parameters": {
    "X_dimension": 20.0,
    "Y_dimension": 30.0,
    "Number_of_Users": 90,
    "Number_of_Routers": 5,
    "X_User_Distribution": "Uniform",
    "Y_User_Distribution": "Uniform",
    "Height": 0.0,
    "K": 0.0,
    "Alpha": 0.0,
    "Transmit_Power": 0,
    "User_Equipment_Height": 0.0
  }
}
,
34.01347041130066


In [70]:
len(no_context)

150

In [72]:
with open("nocontext_12345.txt","w") as file:
    file.write(str(no_context))

In [73]:
for i in range(5):
    print(dataset_list[i]["0"][0])

{'Original_Request': 'In an office setting with dimensions 30m x 40m x 4m, how should I position 6 Nokia 7750 routers to achieve minimum signal degradation among 100 users, given 15 dBm transmit power, path loss parameters K = 0.005 and alpha = 2, with users distributed exponentially along both axes?', 'Parsed_Parameters': {'X_dimension': 30, 'Y_dimension': 40, 'Number_of_Users': 100, 'Number_of_Routers': 6, 'X_User_Distribution': 'Exponential', 'Y_User_Distribution': 'Exponential', 'Height': 4, 'K': 0.005, 'Alpha': 2.0, 'Location': 'Office', 'Device_Name': 'Nokia 7750', 'Transmit_Power': 15}}
{'Original_Request': "Considering a banquet hall with dimensions 50m x 20m and a height of 5m, I'd like to make sure the 150 devices are close to 8 Juniper MX240 routers, each having 25 dBm transmit power, with path loss parameters K = 0.01 and alpha = 3, and user distribution following Gaussian along the X-axis and exponential along the Y-axis.", 'Parsed_Parameters': {'X_dimension': 50, 'Y_dimen